# Curate perturbation dataset with `PerturbationCurator`

Here we use pertpy's `PerturbationCurator` to ensure that a perturbatin dataset conforms to both, `CELLxGENE` (schema 5.1.0) and pertpy's defined criteria.
More specifically, the `PerturbationCurator` builds upon [cellxgene-lamin](https://github.com/laminlabs/cellxgene-lamin) and extends it by further requiring `cell_line` and `X_treatments` columns for the perturbations.

This guide demonstrates how to curate a complex, real world perturbation dataset [McFarland et al. 2020](https://www.nature.com/articles/s41467-020-17440-w) using `PerturbationCurator`. Please have a look at [lamindb's perturbation guide](https://docs.lamin.ai/perturbation) for more details.

In [1]:
#!pip install pertpy[validator]

In [2]:
# Using a local instance here but in practice, we use `laminlabs/pertpy-datasets`
!lamin init --storage ./test-perturbation --schema bionty,wetlab,findrefs

→ connected lamindb: zethson/test-perturbation


In [3]:
import lamindb as ln
import bionty as bt
import wetlab as wl
import findrefs as fr
import pertpy as pt
import scanpy as sc

→ connected lamindb: zethson/test-perturbation


In [4]:
# ln.track("HIRTYxL3aZc70000")

In [5]:
adata = ln.Artifact.using("laminlabs/lamindata").get(uid="Xk7Qaik9vBLV4PKf0001").load()
adata.obs.head(3)

! run input wasn't tracked, call `ln.track()` and re-run


,depmap_id,cancer,cell_det_rate,cell_line,cell_quality,channel,disease,dose_unit,dose_value,doublet_CL1,...,singlet_z_margin,time,tissue_type,tot_reads,nperts,ngenes,ncounts,percent_mito,percent_ribo,chembl-ID
AACTGGTGTCTCTCTG,ACH-000390,True,0.093159,LUDLU-1,normal,nan,lung cancer,µM,0.1,LUDLU1_LUNG,...,12.351139,24,cell_line,787,1,3045,12895.0,3.202792,24.955409,CHEMBL2103875
ATAGGCTCAGATTTCG,ACH-000444,True,0.145728,LU99,normal,2,lung cancer,µM,0.5,LU99_LUNG,...,8.164565,24,cell_line,1597,1,4763,23161.0,7.473771,18.051898,CHEMBL1173655
GCCAAATCAAGCCGTC,ACH-000396,True,0.117330,J82,normal,nan,urinary bladder carcinoma,µM,0.1,J82_URINARY_TRACT,...,11.188513,24,cell_line,1159,1,3834,18062.0,2.762706,22.085040,CHEMBL2028663


In [6]:
# Calculate an embedding because CELLxGENE requires one
sc.tl.pca(adata)

## Curator non-perturbation data

In [7]:
curator = pt.dt.PerturbationCurator(adata, using_key="test-perturbation")  # Fetch all ontologies from this instance
curator.validate()

→ added defaults to the AnnData object: {'assay': 'unknown', 'cell_type': 'unknown', 'development_stage': 'unknown', 'donor_id': 'unknown', 'self_reported_ethnicity': 'unknown', 'suspension_type': 'cell', 'genetic_treatments': '', 'compound_treatments': '', 'environmental_treatments': '', 'combination_treatments': ''}
✓ added 15 records with Feature.name for columns: 'assay', 'cell_type', 'development_stage', 'disease', 'donor_id', 'self_reported_ethnicity', 'sex', 'suspension_type', 'tissue_type', 'organism', 'cell_line', 'genetic_treatments', 'compound_treatments', 'environmental_treatments', 'combination_treatments'
→ validating metadata using registries of instance test-perturbation
• saving validated records of 'var_index'
✓ added 1869 records from public with Gene.ensembl_gene_id for var_index: 'ENSG00000102316', 'ENSG00000109472', 'ENSG00000080007', 'ENSG00000203926', 'ENSG00000232301', 'ENSG00000127419', 'ENSG00000266869', 'ENSG00000108960', 'ENSG00000261316', 'ENSG00000126870'

False

In [8]:
adata.obs["sex"] = adata.obs["sex"].replace({"Unknown": "unknown"})

In [9]:
efo_lo = bt.ExperimentalFactor.public().lookup()

In [10]:
adata.obs["assay"] = efo_lo.single_cell_rna_sequencing.name

In [11]:
adata = adata[:, ~adata.var_names.isin(curator.non_validated["var_index"])].copy()

In [12]:
# Need to recreate Curator object because we are using a new object
curator = pt.dt.PerturbationCurator(adata, using_key="test-perturbation")
curator.validate()

→ validating metadata using registries of instance test-perturbation
• saving validated records of 'var_index'
• saving validated terms of 'assay'
• saving validated terms of 'cell_type'
• saving validated terms of 'development_stage'
• saving validated terms of 'disease'
• saving validated terms of 'donor_id'
• saving validated terms of 'self_reported_ethnicity'
• saving validated terms of 'sex'
• saving validated terms of 'suspension_type'
• saving validated terms of 'tissue_type'
• saving validated terms of 'organism'
• saving validated terms of 'cell_line'
• saving validated terms of 'genetic_treatments'
• saving validated terms of 'compound_treatments'
• saving validated terms of 'environmental_treatments'
• saving validated terms of 'combination_treatments'
✓ var_index is validated against Gene.ensembl_gene_id
✓ assay is validated against ExperimentalFactor.name
• mapping cell_type on CellType.name
!    1 terms is not validated: 'unknown'
→ fix typos, remove non-existent values, 

False

In [13]:
curator.add_new_from("all")

• saving validated records of 'var_index'
• saving validated terms of 'assay'
• saving validated terms of 'cell_type'
✓ added 1 record with CellType.name for cell_type: 'unknown'
• saving validated terms of 'development_stage'
✓ added 1 record with DevelopmentalStage.name for development_stage: 'unknown'
• saving validated terms of 'disease'
• saving validated terms of 'donor_id'
✓ added 1 record with ULabel.name for donor_id: 'unknown'
• saving validated terms of 'self_reported_ethnicity'
✓ added 1 record with Ethnicity.name for self_reported_ethnicity: 'unknown'
• saving validated terms of 'sex'
✓ added 1 record with Phenotype.name for sex: 'unknown'
• saving validated terms of 'suspension_type'
✓ added 1 record with ULabel.name for suspension_type: 'cell'
• saving validated terms of 'tissue_type'
✓ added 1 record with ULabel.name for tissue_type: 'cell_line'
• saving validated terms of 'organism'
• saving validated terms of 'cell_line'
• saving validated terms of 'genetic_treatments

In [14]:
curator.validate()

→ validating metadata using registries of instance test-perturbation
• saving validated records of 'var_index'
• saving validated terms of 'assay'
• saving validated terms of 'cell_type'
• saving validated terms of 'development_stage'
• saving validated terms of 'disease'
• saving validated terms of 'donor_id'
• saving validated terms of 'self_reported_ethnicity'
• saving validated terms of 'sex'
• saving validated terms of 'suspension_type'
• saving validated terms of 'tissue_type'
• saving validated terms of 'organism'
• saving validated terms of 'cell_line'
• saving validated terms of 'genetic_treatments'
• saving validated terms of 'compound_treatments'
• saving validated terms of 'environmental_treatments'
• saving validated terms of 'combination_treatments'
✓ var_index is validated against Gene.ensembl_gene_id
✓ assay is validated against ExperimentalFactor.name
✓ cell_type is validated against CellType.name
✓ development_stage is validated against DevelopmentalStage.name
✓ disea

True

All treatment columns validate but that's only because they're all empty.

## Curate perturbations

In [15]:
# Move 
adata.obs['genetic_treatments'] = adata.obs['perturbation'].where(adata.obs['perturbation_type'] == 'CRISPR', None)
adata.obs['compound_treatments'] = adata.obs['perturbation'].where(adata.obs['perturbation_type'] == 'drug', None)

### Genetic treatments

In [16]:
list(adata.obs["genetic_treatments"].unique())

[nan, 'sggpx4-2', 'sglacz', 'sggpx4-1', 'sgor2j2']

In [17]:
treatments = [
    ("sggpx4-1", "GPX4", "Glutathione Peroxidase 4"),
    ("sggpx4-2", "GPX4", "Glutathione Peroxidase 4"),
    ("sgor2j2", "or2j2", "Olfactory receptor family 2 subfamily J member 2"),
    ("sglacz", "lacz", "beta-galactosidase control"),  # Control from E. coli
]
organism = bt.Organism.lookup().human

genetic_treatments = []
for name, symbol, target_name in treatments:
    treatment = wl.GeneticTreatment(system="CRISPR KO", name=name).save()
    if symbol != "lacz":
        gene_result = bt.Gene.from_source(symbol=symbol, organism=organism)
        gene = gene_result[0] if isinstance(gene_result, list) else gene_result
        gene = gene.save()
    else:
        gene = bt.Gene(symbol=symbol, organism=organism).save()
    target = wl.TreatmentTarget(name=target_name).save()
    target.genes.add(gene)
    treatment.targets.add(target)
    genetic_treatments.append(treatment)

✓ created 1 Gene record from Bionty matching symbol: 'GPX4'
! record with similar name exists! did you mean to load it?


,uid,name,system,sequence,on_target_score,off_target_score,run_id,created_at,created_by_id
id,,,,,,,,,
1,pD8ifwxqVKmz,sggpx4-1,CRISPR KO,None,None,None,None,2024-10-25 13:21:31.828915+00:00,1


→ returning existing TreatmentTarget record with same name: 'Glutathione Peroxidase 4'
✓ created 1 Gene record from Bionty matching synonyms: 'or2j2'
! ambiguous validation in Bionty for 1 record: 'OR2J2'


In [18]:
curator.validate()

→ validating metadata using registries of instance test-perturbation
• saving validated records of 'var_index'
• saving validated terms of 'assay'
• saving validated terms of 'cell_type'
• saving validated terms of 'development_stage'
• saving validated terms of 'disease'
• saving validated terms of 'donor_id'
• saving validated terms of 'self_reported_ethnicity'
• saving validated terms of 'sex'
• saving validated terms of 'suspension_type'
• saving validated terms of 'tissue_type'
• saving validated terms of 'organism'
• saving validated terms of 'cell_line'
• saving validated terms of 'genetic_treatments'
• saving validated terms of 'compound_treatments'
• saving validated terms of 'environmental_treatments'
• saving validated terms of 'combination_treatments'
✓ var_index is validated against Gene.ensembl_gene_id
✓ assay is validated against ExperimentalFactor.name
✓ cell_type is validated against CellType.name
✓ development_stage is validated against DevelopmentalStage.name
✓ disea

False

### Compounds

In [19]:
compounds = wl.Compound.from_values(adata.obs["compound_treatments"], field="name")

✓ created 8 Compound records from Bionty matching name: 'trametinib', 'afatinib', 'dabrafenib', 'gemcitabine', 'navitoclax', 'bortezomib', 'JQ1', 'everolimus'
! did not create Compound records for 6 non-validated names: 'azd5591', 'brd3379', 'control', 'idasanutlin', 'prexasertib', 'taselisib'


In [20]:
# The remaining compounds are not in chebi and we create records for them
for missing in [
    "azd5591",
    "brd3379",
    "control",
    "idasanutlin",
    "prexasertib",
    "taselisib",
]:
    compounds.append(wl.Compound(name=missing))
ln.save(compounds)

In [21]:
drug_metadata = adata.obs[adata.obs["compound_treatments"].notna()]

unique_treatments = drug_metadata[
    ["perturbation", "dose_unit", "dose_value"]
].drop_duplicates()

compound_treatments = []
for _, row in unique_treatments.iterrows():
    compound = wl.Compound.get(name=row["perturbation"])
    treatment = wl.CompoundTreatment(
        name=compound.name,
        concentration=row["dose_value"],
        concentration_unit=row["dose_unit"],
    )
    compound_treatments.append(treatment)

ln.save(compound_treatments)

## References

In [22]:
reference = fr.Reference(
    name="Multiplexed single-cell transcriptional response profiling to define cancer vulnerabilities and therapeutic mechanism of action",
    abbr="McFarland 2020",
    url="https://www.nature.com/articles/s41467-020-17440-w",
    doi="10.1038/s41467-020-17440-w",
    text=(
        "Assays to study cancer cell responses to pharmacologic or genetic perturbations are typically "
        "restricted to using simple phenotypic readouts such as proliferation rate. Information-rich assays, "
        "such as gene-expression profiling, have generally not permitted efficient profiling of a given "
        "perturbation across multiple cellular contexts. Here, we develop MIX-Seq, a method for multiplexed "
        "transcriptional profiling of post-perturbation responses across a mixture of samples with single-cell "
        "resolution, using SNP-based computational demultiplexing of single-cell RNA-sequencing data. We show "
        "that MIX-Seq can be used to profile responses to chemical or genetic perturbations across pools of 100 "
        "or more cancer cell lines. We combine it with Cell Hashing to further multiplex additional experimental "
        "conditions, such as post-treatment time points or drug doses. Analyzing the high-content readout of "
        "scRNA-seq reveals both shared and context-specific transcriptional response components that can identify "
        "drug mechanism of action and enable prediction of long-term cell viability from short-term transcriptional "
        "responses to treatment."
    )
).save()

## Remove unused columns

In [23]:
adata = adata.obs.drop(["depmap_id", "cancer", "cell_quality", "channel", "perturbation",
                        "perturbation_type", "singlet_dev", "singlet_dev_z", "singlet_margin", "singlet_z_margin",
                        "nperts", "ngenes", "ncounts"], axis=1)

## Register curated artifact

In [24]:
artifact = curator.save_artifact(description="McFarland AnnData")

→ validating metadata using registries of instance test-perturbation
• saving validated records of 'var_index'
• saving validated terms of 'assay'
• saving validated terms of 'cell_type'
• saving validated terms of 'development_stage'
• saving validated terms of 'disease'
• saving validated terms of 'donor_id'
• saving validated terms of 'self_reported_ethnicity'
• saving validated terms of 'sex'
• saving validated terms of 'suspension_type'
• saving validated terms of 'tissue_type'
• saving validated terms of 'organism'
• saving validated terms of 'cell_line'
• saving validated terms of 'genetic_treatments'
• saving validated terms of 'compound_treatments'
• saving validated terms of 'environmental_treatments'
• saving validated terms of 'combination_treatments'
✓ var_index is validated against Gene.ensembl_gene_id
✓ assay is validated against ExperimentalFactor.name
✓ cell_type is validated against CellType.name
✓ development_stage is validated against DevelopmentalStage.name
✓ disea

In [25]:
# Set the perturbations and references
artifact.genetic_treatments.set(genetic_treatments)
artifact.compound_treatments.set(compound_treatments)
artifact.references.add(reference)

In [26]:
artifact.describe()

Artifact(uid='OHFkINp7RhR0lADp0000', is_latest=True, description='McFarland AnnData', suffix='.h5ad', type='dataset', size=3498384, hash='4vuqSXa9fXfP7zZPZgA2-g', n_observations=1000, _hash_type='md5', _accessor='AnnData', visibility=1, _key_is_virtual=True, created_at=2024-10-25 13:21:41 UTC)
  Provenance
    .storage = '/home/zeth/PycharmProjects/pertpy-datasets/lamin/test-perturbation'
    .created_by = 'zethson'
  Labels
    .organisms = 'human'
    .cell_types = 'unknown'
    .diseases = 'lung cancer', 'urinary bladder carcinoma', 'colorectal cancer', 'head and neck cancer', 'brain cancer', 'skin cancer', 'gastric cancer', 'esophageal cancer', 'ovarian cancer', 'malignant pancreatic neoplasm', ...
    .cell_lines = 'LUDLU-1', 'LU99', 'J82', 'HCT-15', 'YD-10B', 'SNB75', 'SK-MEL-2', 'UM-UC-1', 'AGS', 'COLO-680N', ...
    .phenotypes = 'male', 'female', 'unknown'
    .experimental_factors = 'single-cell RNA sequencing'
    .developmental_stages = 'unknown'
    .ethnicities = 'unknown